In [1]:
import sys
sys.path.insert(0, '../../')
import keras
import pandas as pd
import numpy as np
import matchzoo as mz

Using TensorFlow backend.


In [2]:
train_pack = mz.datasets.wiki_qa.load_data('train', task='ranking')
valid_pack = mz.datasets.wiki_qa.load_data('dev', task='ranking', filter=True)
predict_pack = mz.datasets.wiki_qa.load_data('test', task='ranking', filter=True)

In [3]:
preprocessor = mz.preprocessors.BasicPreprocessor(fixed_length_left=10, fixed_length_right=40, remove_stop_words=True)
train_pack_processed = preprocessor.fit_transform(train_pack)

Processing text_left with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit: 100%|██████████| 2118/2118 [00:00<00:00, 9070.31it/s]
Processing text_right with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit: 100%|██████████| 18841/18841 [00:03<00:00, 4932.63it/s]
Building VocabularyUnit from a datapack.: 100%|██████████| 404432/404432 [00:00<00:00, 2990917.38it/s]
Processing text_left with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit: 100%|██████████| 2118/2118 [00:00<00:00, 9651.58it/s]
Processing text_right with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit: 100%|██████████| 18841/18841 [00:03<00:00, 5255.00it/s]
Processing length_right with len: 100%|██████████| 18841/18841 [00:00<00:00, 788026.58it/s]


In [4]:
valid_pack_processed = preprocessor.transform(valid_pack)
predict_pack_processed = preprocessor.transform(predict_pack)

Processing text_left with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit: 100%|██████████| 122/122 [00:00<00:00, 9388.91it/s]
Processing text_right with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit: 100%|██████████| 1115/1115 [00:00<00:00, 5325.21it/s]
Processing length_right with len: 100%|██████████| 1115/1115 [00:00<00:00, 625471.31it/s]
Processing text_left with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit: 100%|██████████| 237/237 [00:00<00:00, 9901.49it/s]
Processing text_right with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit: 100%|██████████| 2300/2300 [00:00<00:00, 4386.71it/s]
Processing length_right with len: 100%|██████████| 2300/2300 [00:00<00:00, 725712.72it/s]


In [5]:
ranking_task = mz.tasks.Ranking(loss=mz.losses.RankHingeLoss())
ranking_task.metrics = [
    mz.metrics.NormalizedDiscountedCumulativeGain(k=3),
    mz.metrics.NormalizedDiscountedCumulativeGain(k=5),
    mz.metrics.MeanAveragePrecision()
]

In [30]:
model = mz.models.MatchPyramid()
model.params['input_shapes'] = preprocessor.context['input_shapes']
model.params['task'] = ranking_task
model.params['embedding_input_dim'] = preprocessor.context['vocab_size'] + 1
model.params['embedding_output_dim'] = 300
model.params['embedding_trainable'] = True
model.params['optimizer'] = 'adam'
model.params['dropout_rate'] = 0.1
model.guess_and_fill_missing_params()
model.build()
model.compile()
model.backend.summary()

Parameter "name" set to MatchPyramid.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text_left (InputLayer)          (None, 10)           0                                            
__________________________________________________________________________________________________
text_right (InputLayer)         (None, 40)           0                                            
__________________________________________________________________________________________________
embedding (Embedding)           multiple             5002200     text_left[0][0]                  
                                                                 text_right[0][0]                 
__________________________________________________________________________________________________
matching_layer_5 (MatchingLayer (None, 10, 40, 1)    0           embedd

In [29]:
glove_embedding = mz.datasets.embeddings.load_glove_embedding(dimension=300)
embedding_matrix = glove_embedding.build_matrix(preprocessor.context['vocab_unit'].state['term_index'])

In [31]:
model.load_embedding_matrix(embedding_matrix)

In [32]:
train_generator = mz.DPoolPairDataGenerator(train_pack_processed,
                                            fixed_length_left=10,
                                            fixed_length_right=40,
                                            num_dup=2,
                                            num_neg=1,
                                            batch_size=20)
len(train_generator)

102

In [33]:
predict_generator = mz.DPoolDataGenerator(predict_pack_processed,
                                          fixed_length_left=10,
                                          fixed_length_right=40,
                                          batch_size=20)
len(predict_generator)

118

In [34]:
pred_x, pred_y = predict_generator[:-1]
evaluate = mz.callbacks.EvaluateAllMetrics(model, x=pred_x, y=pred_y, batch_size=len(pred_y))
print(len(pred_y))
print(len(pred_x))
print(pred_x.keys())

2340
7
dict_keys(['id_left', 'text_left', 'length_left', 'id_right', 'text_right', 'length_right', 'dpool_index'])


In [35]:
history = model.fit_generator(train_generator, epochs=20, callbacks=[evaluate], workers=3, use_multiprocessing=True)

Epoch 1/20
102/102 [==============================] - 6s 64ms/step - loss: 0.9475
Validation: loss:1.433169 - normalized_discounted_cumulative_gain@3(0):0.540093 - normalized_discounted_cumulative_gain@5(0):0.611016 - mean_average_precision(0):0.568563
Epoch 2/20
102/102 [==============================] - 5s 45ms/step - loss: 0.5216
Validation: loss:1.501616 - normalized_discounted_cumulative_gain@3(0):0.575958 - normalized_discounted_cumulative_gain@5(0):0.646696 - mean_average_precision(0):0.596191
Epoch 3/20
102/102 [==============================] - 5s 49ms/step - loss: 0.3151
Validation: loss:1.368009 - normalized_discounted_cumulative_gain@3(0):0.573405 - normalized_discounted_cumulative_gain@5(0):0.644039 - mean_average_precision(0):0.594133
Epoch 4/20
102/102 [==============================] - 5s 45ms/step - loss: 0.1867
Validation: loss:1.441315 - normalized_discounted_cumulative_gain@3(0):0.571334 - normalized_discounted_cumulative_gain@5(0):0.647188 - mean_average_precision(